# Fortran magic's documentation

Fortran magic is an [IPython](http://ipython.org) extension that help to use fortran code in an interactive session. 

It adds a `%%fortran` cell magic that compile and import the Fortran code in the cell, using [F2py](http://wiki.scipy.org/F2py).

The contents of the cell are written to a `.f90` file in the
directory `IPYTHONDIR/fortran` using a filename with the hash of the
code. This file is then compiled. The resulting module
is imported and all of its symbols are injected into the user's
namespace.


* Author: Martín Gaitán <gaitan@gmail.com> heavily modeified by John (EBo) David <John.L.David@NASA.gov>
* Homepage: https://github.com/mgaitan/fortran_magic 
* Twitter: [@tin`_`nqn`_`](https://twitter.com/tin_nqn_)
* License: BSD

This software was originally sponsored by [Phasety](http://phasety.com)

Feedback, report of issues and pull requests are welcome!

## Install or upgrade

You can install or upgrade via `pip`

        pip install -U fortran-magic

or directly from the repository using `%install_ext` magic command.

        %install_ext https://raw.github.com/mgaitan/fortran_magic/master/fortranmagic.py
        
or with conda

        conda install -c IOOS fortran-magic
        
Also update SciPy

# Usage 

Then you are ready to load the magic 

In [ ]:
%load_ext fortranmagic

To load it each time IPython starts, list it in your configuration file:

    c.InteractiveShellApp.extensions = [
        'fortranmagic'
    ]

## Basic example

Just mark the cell with `%%fortran` in the first line. The code will be highlighted accordingly and compiled when the cell is run 

In [ ]:
%%fortran

subroutine f1(x, y, z)
    real, intent(in) :: x,y
    real, intent(out) :: z

    z = sin(x+y)

end subroutine f1

In [ ]:
f1(1.0, 2.1415)

In [ ]:
print f1.__doc__

## Verbosity

By default the magic only returns output when the compilation process fails. But you can increase the verbosity with the flag `-v`

In [ ]:
%%fortran -v 

module hi
  integer :: five = 5
end module   

In [ ]:
%%fortran -vv 

module hi
  integer :: five = 5
end module   

In [ ]:
%%fortran -vvv

module hi
  integer :: five = 5
end module   

## Using f2py options

Almost all f2py's command line options are exposed to the `%%fortran` cell magic. See the docstring for detail. For example:

In [ ]:
%%fortran --fcompiler gnu95 --compiler unix --f90flags "-d" --noarch
C
      SUBROUTINE ZADD(A,B,C,N)
C
      DOUBLE COMPLEX A(*)
      DOUBLE COMPLEX B(*)
      DOUBLE COMPLEX C(*)
      INTEGER N
      DO 20 J = 1, N
         C(J) = A(J)+B(J)
 20   CONTINUE
      END


In [ ]:
print(zadd.__doc__)

## Linking resources

Use `--link` option. This is `--link-<resource>` in f2py command line

In [ ]:
%%fortran --link lapack -vv

subroutine solve(A, b, x, n)
    ! solve the matrix equation A*x=b using LAPACK
    implicit none

    real*8, dimension(n,n), intent(in) :: A
    real*8, dimension(n), intent(in) :: b
    real*8, dimension(n), intent(out) :: x

    integer :: i, j, pivot(n), ok

    integer, intent(in) :: n
    x = b

    ! find the solution using the LAPACK routine SGESV
    call DGESV(n, 1, A, n, pivot, x, n, ok)
    
end subroutine

In [ ]:
import numpy as np
A = np.array([[1, 2.5], [-3, 4]])
b = np.array([1, 2.5])

In [ ]:
solve?

Which is, by the way, the same than

In [ ]:
np.linalg.solve(A, b)

## Extra arguments

F2py could have many other arguments. You could append extra arguments with `--extra`. For example:


      %%fortran --extra '-L/path/to/open/ -lopenblas'
      
      %%fortran --extra '-D<define> -U<name>'
   
      %%fortran --extra '-DPREPEND_FORTRAN -DUPPERCASE_FORTRAN'
      
The option `--extra` could be given multiple times.

## Save options

By default, `%%fortran` call to `f2py` without parameters (except, of course, the `-m` and `-c` needed to compile a new module). You can change this behaviour with `%fortran_config`. This line magic can be used in three different ways:


    %fortran_config

        Show the current custom configuration

    %fortran_config --defaults

        Delete the current configuration and back to defaults

    %fortran_config <other options>

        Save (persitently) <other options> to use with %%fortran. The same arguments allowed for `%%fortran` are available
        
For example, to set the highest verbose level (`-vvv`) and `gnu95` as default `--fcompiler`:

In [ ]:
%fortran_config -vvv --fcompiler gnu95

Now the use of `%%fortran` will include `-vvv --fcompiler gnu95` implicitly

In [ ]:
%%fortran

module hi
  integer :: five = 5
end module   

We can see whatever the default config has

In [ ]:
%fortran_config

You can override that global configuration for one specific cell. For example, `%%fortran -vv` will change the the verbose level but still use `--fcompiler gnu95`

In [ ]:
%%fortran -vv

module hi
  integer :: five = 5
end module   

To clear the custom defaults and back to the defaults (no arguments) use:

In [ ]:
%fortran_config --defaults

## Help on f2py 

F2py has some flag that output help. See the docstring of `%f2py_help`

In [ ]:
%f2py_help --link blas

In [ ]:
%f2py_help --fcompiler

In [ ]:
%f2py_help --compiler

## Creat a file and compiling...

### With Fortran

examples taken from https://www.wakari.io/nb/url///wakari.io/static/notebooks/Lecture_6A_Fortran_and_C.ipynb

In [ ]:
%%file hellofortran.f
C File  hellofortran.f
        subroutine hellofortran (n)
        integer n
       
        do 100 i=0, n
            print *, "Fortran says hello"
100     continue
        end

In [ ]:
!f2py -c -m hellofortran hellofortran.f

In [ ]:
%%file hello.py
import hellofortran

hellofortran.hellofortran(5)

In [ ]:
# run the script
!python hello.py

### With C

In [ ]:
%%file functions.c

#include <stdio.h>

void hello(int n);

void
hello(int n)
{
    int i;
    
    for (i = 0; i < n; i++)
    {
        printf("C says hello\n");
    }
}


In [ ]:
!gcc -c -Wall -O2 -Wall -ansi -pedantic -fPIC -o functions.o functions.c
!gcc -o libfunctions.so -shared functions.o

In [ ]:
!file libfunctions.so

In [ ]:
%%file functions.py

import numpy
import ctypes

_libfunctions = numpy.ctypeslib.load_library('libfunctions', '.')

_libfunctions.hello.argtypes = [ctypes.c_int]
_libfunctions.hello.restype  =  ctypes.c_void_p

def hello(n):
    return _libfunctions.hello(int(n))

In [ ]:
%%file run_hello_c.py

import functions

functions.hello(3)

In [ ]:
# run the script
!python run_hello_c.py

# Using Cython to compile code into C



In [ ]:
%load_ext Cython

In [ ]:
%%cython

cdef int a = 0
for i in range(10):
    a += i
print a

In [ ]:
%%cython --annotate
def fib(long n):
    cdef long a=0, b=1, i
    for i in range(n):
        a, b = a + b, a
    return a


In [ ]:
fib(12)

---------------

* Bugs? Ideas? [Open an issue](https://github.com/mgaitan/fortran_magic)
* Do you want to collaborate? [Fork it](https://github.com/mgaitan/fortran_magic/fork)! and send a pull-request